In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)

In [90]:
%load_ext autoreload
%autoreload 2
from functions import loaddata, MACD, RSI
from portfolio import Portfolio

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return 0
    else:
        return 2
    
def my_log(x):
    if x != 0 and np.isnan(x) == False:
        return np.log(x)
    else:
        return x

In [92]:
start_date = 20180101
end_date = 20190101
rtxm_ti2 = loaddata("rtxm_ti2", start_date, end_date)
rtxm_ti1 = loaddata("rtxm_ti1", start_date, end_date)
r_ti1 = loaddata("r_ti1", start_date, end_date)
r_ti2 = loaddata("r_ti2", start_date, end_date)
volume = loaddata("volall_day", start_date, end_date)
mid_open = loaddata("mid_open", start_date, end_date)
mid_close = loaddata("mid_close", start_date, end_date)

start_date1 = 20190101
end_date1 = 20200101
rtxm_ti2_test = loaddata("rtxm_ti2", start_date1, end_date1)
rtxm_ti1_test = loaddata("rtxm_ti1", start_date1, end_date1)
r_ti1_test = loaddata("r_ti1", start_date1, end_date1)
r_ti2_test = loaddata("r_ti2", start_date1, end_date1)
volume_test = loaddata("volall_day", start_date1, end_date1)
mid_close_test = loaddata("mid_close", start_date1, end_date1)
mid_open_test = loaddata("mid_open", start_date1, end_date1)

total = []
for i in range(mid_close.shape[0]):
    price_i = mid_close.iloc[i]

    overnight_i = rtxm_ti1.iloc[i, 1:]
    intraday_i = rtxm_ti2.iloc[i,:-1]
    volume_i = volume.iloc[i, :-1].map(lambda x: my_log(x), na_action="ignore")

    rsi_12 = RSI(rtxm_ti2.iloc[i], 12)[:-1]

    macd_26_12_9 = MACD(price_i, 26, 12, 9)[:-1]
   
    y = r_ti2.iloc[i, 1:].map(lambda x: sign(x), na_action="ignore")

    data = {'X1': np.array(overnight_i), 'X2': np.array(intraday_i), "X3": np.array(volume_i), 
            "X4": np.array(rsi_12), "X5": np.array(macd_26_12_9),
            "Y": np.array(y)}

    data_train = pd.DataFrame(data)
    new_train = data_train.dropna()

    if new_train.shape[0] > 100:
        total.append(new_train)

final_df = pd.concat(total)

LR = LogisticRegression(max_iter=1000)
temp = final_df.sample(frac=1).reset_index(drop=True)
final_x_training = temp[["X1","X2","X3","X4","X5"]]
final_y_training = temp[["Y"]]
attempt = poly.fit_transform(final_x_training)
LR.fit(attempt, np.ravel(final_y_training))

total = []
test = []
pls = []
chk = []
for i in range(mid_close_test.shape[0]):
    total_made = 0
    
    price_i_test = mid_close_test.iloc[i]
    open_price_i_test = mid_open_test.iloc[i]
    overnight_i_test = rtxm_ti1_test.iloc[i, 1:]
    intraday_i_test = rtxm_ti2_test.iloc[i,:-1]
    volume_i_test = volume_test.iloc[i, :-1].map(lambda x: my_log(x), na_action="ignore")

    rsi_12_test = RSI(rtxm_ti2_test.iloc[i], 12)[:-1]

    macd_26_12_9_test = MACD(price_i_test, 26, 12, 9)[:-1]

    y_test = r_ti2_test.iloc[i, 1:].map(lambda x: sign(x), na_action="ignore")

    data_test = {'X1': np.array(overnight_i_test), 'X2': np.array(intraday_i_test), "X3": np.array(volume_i_test), 
        "X4": np.array(rsi_12_test), "X5": np.array(macd_26_12_9_test),
        "Y": np.array(y_test)}

    data_test = pd.DataFrame(data_test, index=y_test.index)

    new_test = data_test.dropna()
    X_test = new_test[["X1","X2","X3","X4","X5"]]  
    y_test = new_test[["Y"]]
    
    if X_test.shape[0] > 100:
        x_test = poly.fit_transform(X_test)
        chk.append(LR.score(x_test, y_test))
        signals = LR.predict(x_test)
        count = 0

        signal_dates = new_test.index

        for j in range(len(signals)):
            price1 = open_price_i_test.loc[signal_dates[j]]
            price2 = price_i_test.loc[signal_dates[j]]
            pls.append(int(price1 - price2 < 0) == np.array(y_test)[j][0])
            #Short
            if signals[j] == 0:
                gain = price1 - price2
                test.append(gain > 0)
                total_made += gain
            #Long
            elif signals[j] == 1:
                gain = price2 - price1
                test.append(gain > 0)
                total_made += gain
        total.append(total_made)

print("Mean Score: ", sum(chk)/len(chk))
print("Max Score: ", max(chk))
print("Min Score: ", min(chk))
print("Proportion of correct labels: ", sum(pls)/len(pls))
print("Proportion of correct guesses: ", sum(test)/len(test))
print("Total Money Made: ", sum(total))
print("Initial Funds: ", mid_open_test.iloc[:,0].dropna().sum())
print("YoY: ", sum(total) / mid_open_test.iloc[:,0].dropna().sum())


/Users/jameshetherington/anaconda3/envs/ml-0451/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Mean Score:  0.5238093576955263
Max Score:  0.6584362139917695
Min Score:  0.39090909090909093
Proportion of correct labels:  0.9472727502676028
Proportion of correct guesses:  0.5058228773717777
Total Money Made:  11999.027628168433
Initial Funds:  40712.52499175072
YoY:  0.2947257049421452


In [72]:
LR.coef_

array([[ 2.02891295e-06,  4.09201138e-07,  1.07148323e-07,
         2.14357293e-05,  4.65322478e-05, -3.88921596e-07,
         4.85051079e-09,  1.26894515e-10,  5.85301728e-06,
         1.93706770e-05,  2.65476606e-09, -3.56003423e-09,
         2.03914192e-06,  3.48452847e-06,  1.84046148e-08,
         1.79189035e-04,  3.55432274e-04, -3.98862449e-06,
         8.63069045e-04, -1.13640238e-05,  1.86320594e-06,
         4.68513525e-11,  1.85071077e-11,  7.38440934e-08,
         2.46809448e-07, -1.46214582e-10,  4.29382119e-11,
        -1.59870601e-10,  1.88688025e-08, -3.58295965e-10,
         8.38349874e-05,  2.75469656e-04,  4.57001784e-08,
         9.83866180e-04,  1.29693989e-07,  2.50040375e-09,
         1.72538666e-10, -5.80634317e-08, -1.74981580e-07,
         9.06822630e-10,  3.77726977e-05,  7.50143117e-05,
         2.71736013e-07,  1.00303058e-04,  1.27713965e-06,
        -1.20127231e-08,  3.78008156e-04, -1.12542597e-04,
        -3.09821568e-05, -2.62195120e-05, -7.91698368e-0

1